Copyright (C) 2019 Software Platform Lab, Seoul National University

Licensed under the Apache License, Version 2.0 (the "License"); 

you may not use this file except in compliance with the License. 

You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 

Unless required by applicable law or agreed to in writing, software 

distributed under the License is distributed on an "AS IS" BASIS, 


WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. 


See the License for the specific language governing permissions and


limitations under the License.

In [1]:
%env CUDA_VISIBLE_DEVICES=3

env: CUDA_VISIBLE_DEVICES=3


In [5]:
!gpustat

ds1.snu.ac.kr  Tue Nov  5 17:30:10 2019
[0] TITAN Xp         | 26'C,   0 % |     0 / 12196 MB |
[1] TITAN Xp         | 30'C,   0 % |   167 / 12196 MB | ds4user3(157M)
[2] TITAN Xp         | 25'C,   0 % |     0 / 12196 MB |
[3] TITAN Xp         | 31'C,   0 % |     0 / 12196 MB |
[4] TITAN Xp         | 29'C,   0 % | 11849 / 12196 MB | ds4user5(11839M)
[5] TITAN Xp         | 31'C,   0 % |     0 / 12196 MB |
[6] TITAN Xp         | 27'C,   0 % |   167 / 12196 MB | ds4user7(157M)
[7] TITAN Xp         | 31'C,   0 % |     0 / 12196 MB |


### Preparing MNIST Dataset

In [ ]:
import tensorflow as tf

# Download the mnist dataset using keras
data_train, data_test = tf.keras.datasets.mnist.load_data()

# Parse images and labels
(train_images, train_labels) = data_train
(test_images, test_labels) = data_test

# Numpy reshape & type casting
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1).astype('float32')
train_labels = train_labels.astype('int64')
test_labels = test_labels.astype('int64')


# Normalizing the images to the range of [0., 1.]
train_images /= 255.
test_images /= 255.

print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)

## 0. Recap

Build simple CNN by using TensorFlow `layers` API.

In [ ]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
    train_ds = train_ds.shuffle(10000)
    train_ds = train_ds.batch(batch_size)

    test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
    test_ds = test_ds.batch(batch_size)

    # create an empty iterator with only type and shape information
    iterator = tf.data.Iterator.from_structure(tf.compat.v1.data.get_output_types(train_ds),
                                               tf.compat.v1.data.get_output_shapes(train_ds))
    x, y = iterator.get_next()

    # create an operation for initializing the iterator with the train or test dataset
    train_init = iterator.make_initializer(train_ds)
    test_init = iterator.make_initializer(test_ds)

    x = tf.layers.conv2d(x, 32, [3, 3], activation='relu', padding='same')
    x = tf.layers.max_pooling2d(x, [2, 2], [2, 2], padding='same')

    x = tf.layers.conv2d(x, 64, [3, 3], activation='relu', padding='same')
    x = tf.layers.max_pooling2d(x, [2, 2], [2, 2], padding='same')

    x = tf.layers.conv2d(x, 128, [3, 3], activation='relu', padding='same')
    x = tf.layers.max_pooling2d(x, [2, 2], [2, 2], padding='same')

    x = tf.layers.flatten(x)
    x = tf.layers.dense(x, 256, activation='relu')
    logits = tf.layers.dense(x, 10)

    loss = tf.reduce_mean(tf.losses.sparse_softmax_cross_entropy(logits=logits, labels=y))
    train_op = tf.train.AdamOptimizer(0.001).minimize(loss)
    num_correct_preds = tf.reduce_sum(tf.cast(tf.equal(tf.argmax(logits, 1), y), tf.int32))

    init = tf.global_variables_initializer()

with tf.Session(graph=graph) as sess:
    sess.run(init)

    for epoch in range(10):
        sess.run(train_init)
        total_loss = 0
        total_corrects = 0

        try:
            while True:
                _, loss_val, corrects_val = sess.run([train_op, loss, num_correct_preds])
                total_loss += loss_val
                total_corrects += corrects_val
        except tf.errors.OutOfRangeError:
            pass

        print('Epoch: %02d' % (epoch + 1),
              'Loss = {:2.4f}'.format(total_loss * batch_size / 60000),
              'Train accuracy = {:2.4f}'.format(total_corrects / 60000))

    sess.run(test_init)
    total_corrects = 0

    try:
        while True:
            corrects_val = sess.run(num_correct_preds)
            total_corrects += corrects_val
    except tf.errors.OutOfRangeError:
        pass

    print('Test accuracy = {:2.4f}'.format(total_corrects / 10000))

# TF High Level APIs
TensorFlow provides high level APIs to define and execute neural network more easily. In this session, we will take a look at two of them: Keras and Eager execution.

## 1. Keras (tf.keras)
Keras is a high-level API to build and train deep learning models. It's used for fast prototyping, advanced research, and production, with three key advantages:

- **User friendly**: Keras has a simple, consistent interface optimized for common use cases. It provides clear and actionable feedback for user errors.
- **Modular and composable**: Keras models are made by connecting configurable building blocks together, with few restrictions.
- **Easy to extend**: Write custom building blocks to express new ideas for research. Create new layers, loss functions, and develop state-of-the-art models.



### Defining a model
Keras API is accessible in the `tf.keras` namespace. Let's take a look how we can define a model using Keras API.


In [ ]:
#keras는 최신 텐서플로 api, data set api와 인터렉션이 편해짐
#확장성쉽고, 사용도 편하고..
import tensorflow as tf
from tensorflow.keras import layers

# Let's build a stack of *sequential* layers, which is
# the most common form of neural network graphs.
model = tf.keras.Sequential()
#여러개의 모델 빈껍데기 생성

# Adds a reshaping layer that transforms (28, 28, 1) to (784,)
model.add(layers.Reshape((784,), input_shape=(28, 28, 1)))
#원하는 레이어 생성 28x28x1을 781로 바꿈

# Adds a dense layer with 128 units to the model
model.add(layers.Dense(units=128, activation='relu'))
# 784짜리 유닛에 128짜리 유닛으로 바꾸는 dense activation 씀


# Adds another layer, which has L2 regularization applied to the kernel matrix
model.add(layers.Dense(units=64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
#커널에 L2 regul 사용 너무 복잡해지지 않도록 쓰는 방법


# Adds a dense layer with 10 output units
model.add(layers.Dense(units=10, activation='linear'))
#마지막으로 10개의 클래스로 학습해야하니 linear한 dense layer추가 

If you visit https://www.tensorflow.org/api_docs/python/tf/keras/layers, you can find the supported layers, for example, Conv2D, BatchNormalization, LSTM, MaxPool, etc.

### Setting up training
After the model is constructed, `compile` method configures how to learn the model, which allows us to specify the following:
* `optimizer`: This field specifies which optimizer to use. We can pass an optimizer instance (e.g., `tf.train.AdamOptimizer`, `tf.train.RMSPropOptimizer`), which are defined in  `tf.train` module.
* `loss`: The function to minimize during optimization. Common choices include `mean square error (mse)`, `[categorical|binary]_crossentropy`. Loss functions are specified by name or by passing a callable object from the `tf.keras.losses` module.
* `metrics`: Used to monitor training. We can put string names or callables defined in `tf.keras.metrics` module (e.g. `'accuracy'`)

In [ ]:
model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
#metircs에 accuracy면 accuracy가 얼마인지 보여준다.
#모델이 전부 준비가 된것

### Training a model
We can train the model using the `fit` method and then the model is "fit" to the training data. We can specify the training data to use (`images_train` and `labels_train`), how many epochs we will run (`epochs`), and how many items to be processed in a batch (`batch_size`).

In [ ]:
model.fit(train_images, train_labels, epochs=10, batch_size=128)
#트레이닝 폭 10번

### Evaluating the model
Finally, we evaluate the trained model using test dataset.

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('Test accuracy:', test_acc)

### Quiz 1.
First, define a multi-layer model following the CNN model defined above using `tf.layers` API.
The model comprises 3 convolutional layers, 3 max pooling layers, and 1 dense layer.

In [ ]:
from tensorflow.keras import layers

############# Write here. #############
# model = tf.keras.Sequential()
# model.add(...)


#######################################

Using the model and `(image, labels)` above, let's train the model using the following configuration:
* optimizer: `tf.train.AdamOptimizer`
* learning rate: 0.001
* loss: `SparseCategoricalCrossentropy`
* metrics: `accuracy`
* batch size: 128
* epochs: 10

Then, change the model to employ l2 regualrization loss on weights using coefficient `1e-4`.
Let's check whether the regualrization improves the accuracy or not.

In [ ]:
############# Write here. #############
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=10, batch_size=128)
#######################################

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print('Test accuracy:', test_acc)

## 2. Eager execution

Eager execution is a flexible machine learning platform for research and experimentation, providing:

* **An intuitive interface** Structure your code naturally and use Python data structures. Quickly iterate on small models and small data.
* **Easier debugging** Call ops directly to inspect running models and test changes. Use standard Python debugging tools for immediate error reporting.
* **Natural control flow** Use Python control flow instead of graph control flow, simplifying the specification of dynamic models.

Eager execution supports most TensorFlow operations and GPU acceleration. For a collection of examples running in eager execution, see: tensorflow/contrib/eager/python/examples.






### Setup

To start eager execution, add `tf.enable_eager_execution()`
 to the **beginning** of the program or console session. In the jupyter (or colab) environment, you need to restart the runtime if the session has executed any command.

In [3]:
#그래프를 정의하고 차후에 수행하는 방식 과 다르게 파이선 코드레벨에서 적용되어 바로 진행되는 방식. 직관적
#파이선에 기본적으로 있는 pdb를 바로 쓸수있음 디버깅등
import tensorflow as tf
tf.enable_eager_execution()

In [4]:
# Checks whether the eager mode is used (True if enabled)
tf.executing_eagerly()

True

Unlike in the graph mode, all operations (e.g., constant, matrix multiplication) are returned directly. Recall that the operators are not executed until `Session.run()` and thus the values are not available even though we run statement in Python level.

### Constants
Instead, in the eager execution, the values are returned immediately. Let's start with constants that we defined at the previous sessions.

In [6]:
#그래프를 수행하는 sess run이 없어짐
#name은 의미 없음
# constant of 1d tensor, or a vector
a = tf.constant([2,2], name = 'vector')

# constant of 2x2 tensor, or a matrix
b = tf.constant([[0,2], [1,3]], name = 'matrix')

print(a)
print(b)

# See what happens if you uncomment the last line
# print(a.name)

tf.Tensor([2 2], shape=(2,), dtype=int32)
tf.Tensor(
[[0 2]
 [1 3]], shape=(2, 2), dtype=int32)


We can see the results are printed. Obviously, the `name` field is not useful since we can now access the values in python code, so uncommenting the last line raises an error in `Tensor.name`.

As you might notice, this way is simpler and more intuitive than in the graph mode, where the value can be retrieved only when `session.run()` is called as follows:

In [7]:
# In the graph mode, print(aa) prints only the metadata of aa
# aa라는 것에 그래프를 정의할 때 eager에서는 [2,2]라는게 값을 갖고 있는게 차이점
graph = tf.Graph()
with graph.as_default():
    aa = tf.constant([2,2], name = 'vector')
print(aa)
  
# The value of the aa can be obtained through sess.run(aa)
with tf.Session(graph=graph) as sess:
    print(sess.run(aa))

Tensor("vector:0", shape=(2,), dtype=int32)
[2 2]


### Math Ops
Eager execution also allows the mathmatical operations to be computed immediately. Let's take the same examples that we saw with the graph mode.

In [9]:
div = tf.div(b, a)
print(div)
# sess.run할필요 없이 바로 수행가능
# See what happens if you uncomment the last line
# print(div.op) #에러뜸 operation없다

tf.Tensor(
[[0 1]
 [0 1]], shape=(2, 2), dtype=int32)


Even better, TF eager execution supports operator overloading and numpy operations. See how the computation becomes easier.

In [10]:
# Operator overloading
print(b/a)

# Numpy integration
import numpy as np
print(np.divide(b, a))

tf.Tensor(
[[0.  1. ]
 [0.5 1.5]], shape=(2, 2), dtype=float64)
[[0.  1. ]
 [0.5 1.5]]


### Quiz 2.
Define three 2x2 matrices (A, B, C) with the following values, and print the value of A x B + C (use eager execution)
* A: [[3, 4], [2, 1]]
* B: [[1,2], [1,2]]
* C: [[-1,1],[4,1]]

In [12]:
########## Write here. ##########
a = tf.constant([3,4], [2,1])
b = tf.constant([1,2], [1,2])
c = tf.constant([-1,1], [4,1])
prin(axb+c)

##############################

TypeError: unhashable type: 'list'

### Dynamic control flow

A major benefit of eager execution is that all the functionality of the host language is available while your model is executing. So, for example, it is easy to write fizzbuzz game where any number divisible by three is replaced with the word "fizz", and any number divisible by five is replaced with the word "buzz" (similar to the 3-6-9 game).

In [13]:
# Native python code
def fizzbuzz(max_num):
    counter = 0
    for num in range(1, max_num+1):
        if int(num % 3) == 0 and int(num % 5) == 0:
            print('FizzBuzz')
        elif int(num % 3) == 0:
            print('Fizz')
        elif int(num % 5) == 0:
            print('Buzz')
        else:
            print(num)
        counter += 1
    
fizzbuzz(20)

1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14
FizzBuzz
16
17
Fizz
19
Buzz


In eager execution, we need to add minor changes in a few lines.

In [14]:
def fizzbuzz_eager(max_num):
    counter = tf.constant(0) # counter = 0
    max_num = tf.convert_to_tensor(max_num) #
    for num in range(1, max_num.numpy() + 1): #
        num = tf.constant(num) # 
        if int(num % 3) == 0 and int(num % 5) == 0:
            print('FizzBuzz')
        elif int(num % 3) == 0:
            print('Fizz')
        elif int(num % 5) == 0:
            print('Buzz')
        else:
            print(num.numpy()) # print(num)
        counter += 1
    
fizzbuzz_eager(20)

1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14
FizzBuzz
16
17
Fizz
19
Buzz


Implementing the same thing using TensorFlow graph mode: [Tensorflow FizzBuzz Revisited (Ricky Han blog)](https://rickyhan.com/jekyll/update/2018/02/16/tensorflow-fizzbuzz-revisited.html)

In [15]:
import tensorflow as tf

def fizzbuzz_graph(max_num):
    # Define variable and while_loop
    graph = tf.Graph()
    with graph.as_default():
        arr = tf.Variable([str(i) for i in range(1, max_num+1)])
        # nasty tf.while_loop and tf.cond ops
        while_op = tf.while_loop(
            (lambda i, _: tf.less(i, max_num+1)), 
            (lambda i, _: (tf.add(i,1), tf.cond(
                tf.logical_and(tf.equal(tf.mod(i, 3), 0), tf.equal(tf.mod(i, 5), 0)),
                (lambda : tf.assign(arr[(i - 1)], 'FizzBuzz')),
                (lambda : tf.cond(tf.equal(tf.mod(i, 3), 0),
                    (lambda : tf.assign(arr[(i - 1)], 'Fizz')),
                    (lambda : tf.cond(tf.equal(tf.mod(i, 5), 0),
                        (lambda : tf.assign(arr[(i - 1)], 'Buzz')),
                        (lambda : arr)))))))),
            [1, arr])

    # Call Session.run()
    with tf.Session(graph = graph) as sess:
        sess.run(tf.global_variables_initializer())
        idx, array = sess.run(while_op)
        print(array)


fizzbuzz_graph(100)

[b'1' b'2' b'Fizz' b'4' b'Buzz' b'Fizz' b'7' b'8' b'Fizz' b'Buzz' b'11'
 b'Fizz' b'13' b'14' b'FizzBuzz' b'16' b'17' b'Fizz' b'19' b'Buzz' b'Fizz'
 b'22' b'23' b'Fizz' b'Buzz' b'26' b'Fizz' b'28' b'29' b'FizzBuzz' b'31'
 b'32' b'Fizz' b'34' b'Buzz' b'Fizz' b'37' b'38' b'Fizz' b'Buzz' b'41'
 b'Fizz' b'43' b'44' b'FizzBuzz' b'46' b'47' b'Fizz' b'49' b'Buzz' b'Fizz'
 b'52' b'53' b'Fizz' b'Buzz' b'56' b'Fizz' b'58' b'59' b'FizzBuzz' b'61'
 b'62' b'Fizz' b'64' b'Buzz' b'Fizz' b'67' b'68' b'Fizz' b'Buzz' b'71'
 b'Fizz' b'73' b'74' b'FizzBuzz' b'76' b'77' b'Fizz' b'79' b'Buzz' b'Fizz'
 b'82' b'83' b'Fizz' b'Buzz' b'86' b'Fizz' b'88' b'89' b'FizzBuzz' b'91'
 b'92' b'Fizz' b'94' b'Buzz' b'Fizz' b'97' b'98' b'Fizz' b'Buzz']


### Eager training

Automatic differentiation is useful for implementing machine learning algorithms such as backpropagation for training neural networks. During eager execution, use `tf.GradientTape` to trace operations for computing gradients later.

`tf.GradientTape` is an opt-in feature to provide maximal performance when not tracing. Since different operations can occur during each call, all forward-pass operations get recorded to a "tape". To compute the gradient, play the tape backwards and then discard. A particular `tf.GradientTape` can only compute gradients once; subsequent calls throw a runtime error.



In [16]:
#eager로 execution할수있는데, tarin은 backpropagation으로 gradient로 계산해야됨.
#tensor에서는 earger로 forwoard로 다 기록이됨. 꺼꾸로 가면서 그레디언트 계산
w = tf.Variable([[1.0, 2.0]])
with tf.GradientTape() as tape:
    loss = w * w #relu등이 여기서 수행됨

grad = tape.gradient(loss, w) #처음 뭐에 gradint를 뽑을거냐.
print(grad)  # => tf.Tensor([[2. 4.]], shape=(1, 2), dtype=float32)

tf.Tensor([[2. 4.]], shape=(1, 2), dtype=float32)


### Model Subclassing
We can build a fully-customizable model by subclassing [tf.keras.Model](https://www.tensorflow.org/api_docs/python/tf/keras/models/Model) and defining your own forward pass. Layers are created in the `__init__` method and they are set as attributes of the class instance. The forward pass is defined in the `call` method. Model subclassing is particularly useful when eager execution is enabled since you don't have to worry about keeping track of your variables. You can always access them by `model.trainable_variables`.

Below is an example of a linear regression model to be defined as a subclass of `tf.keras.Model`, and then be trained using loss and gradient function, which is defined imperatively.

In [17]:
NUM_EXAMPLES = 2000
toy_inputs = tf.random_normal([NUM_EXAMPLES, 1])
noise = tf.random_normal([NUM_EXAMPLES, 1])
toy_outputs = toy_inputs * 2 - 1 + noise * 1/4

#linear regression모델

In [18]:


class ToyModel(tf.keras.Model):
    def __init__(self):
        """Define layers"""
        super(ToyModel, self).__init__()
        self.dense = tf.keras.layers.Dense(units=1) #tf. variable들을 정의

    def call(self, input):
        """Define forward pass."""
        result = self.dense(input)        
        return result #어떤 연산이 적용되었는지. cnn은 class cnn tf.keras모델을 부모르 가져오고 self.conv1, conv2, conv3정의하고 fully conected layer정의
    #self.max1, max2 다음 layer 계산할수있도록 적용
    #여기는 dense layer 1개임

# The loss function to be optimized (MSE loss)
def loss(model, inputs, targets):
    error = model(inputs) - targets
    return tf.reduce_mean(tf.square(error))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)

model = ToyModel()
print("Initial loss: {:.3f}".format(loss(model, toy_inputs, toy_outputs)))
print("Trainable variables:", model.trainable_variables) #어떤 varialbe이 tariner variable접근
#크기하나짜리 커널 shape1x1 등ㅇ이 모델에 존재하는 것을 확인할수있음

Initial loss: 8.843
Trainable variables: [<tf.Variable 'toy_model/dense/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[-0.84334433]], dtype=float32)>, <tf.Variable 'toy_model/dense/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


In [21]:
# Training loop
for i in range(300):
    with tf.GradientTape() as tape:
        loss_value = loss(model, toy_inputs, toy_outputs)
    grads = tape.gradient(loss_value, model.trainable_variables) #loss funcion이 그레디언트를 desnce layer의 커널과 바이어스로 그레디언트
    optimizer.apply_gradients(zip(grads, model.trainable_variables), 
                              global_step=tf.train.get_or_create_global_step()) #위에 그레디언트로 optimizer ㄱㄱ, 기존은 sess.run으로 수행
    if i % 20 == 0:
        print("Loss at step {:03d}: {:.3f}".format(i, loss(model, toy_inputs, toy_outputs)))

print("Final loss: {:.3f}".format(loss(model, toy_inputs, toy_outputs)))
print("Trainable variables:", model.trainable_variables)

Loss at step 000: 0.065
Loss at step 020: 0.065
Loss at step 040: 0.065
Loss at step 060: 0.065
Loss at step 080: 0.065
Loss at step 100: 0.065
Loss at step 120: 0.065
Loss at step 140: 0.065
Loss at step 160: 0.065
Loss at step 180: 0.065
Loss at step 200: 0.065
Loss at step 220: 0.065
Loss at step 240: 0.065
Loss at step 260: 0.065
Loss at step 280: 0.065
Final loss: 0.065
Trainable variables: [<tf.Variable 'toy_model/dense/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[1.9967932]], dtype=float32)>, <tf.Variable 'toy_model/dense/bias:0' shape=(1,) dtype=float32, numpy=array([-0.9994885], dtype=float32)>]


It's not required to set an input shape for the `tf.keras.Model` class since the parameters are set the first time input is passed to the layer.

tf.keras.layers classes create and contain their own model variables that are tied to the lifetime of their layer objects. To share layer variables, share their objects.

Below examples shows a new model that relies on the previous toy model. We are going to employ an additional bias to fit a slightly different data.

In [22]:
#이 모델을 가져와서 데이타셋을 다르게 쓰는.. 모든 값이 +3이됨/ -1에 +3이라 다른 파라미터 모델필요
#바이어스 추가 한다음에 그것만 학습
toy_outputs_2 = toy_outputs + 3

class ToyModel2(tf.keras.Model):
    def __init__(self, toy_model): #아까만든 토이모델
        """Define layers"""
        super(ToyModel2, self).__init__()
        self.toy_model = toy_model
        self.b = tf.Variable(0., name='another_bias') #0으로 초기화된 다른 바이어스로 적용

    def call(self, input):
        """Define forward pass."""
        result = self.toy_model(input)        
        return result + self.b #바이어스 추가


model2 = ToyModel2(model) #초기값 받는다
print("Initial loss: {:.3f}".format(loss(model2, toy_inputs, toy_outputs_2))) # 3x3이라서 9의 에러를 볼수있음
print("Trainable variables:", model2.trainable_variables)


#Trainable variables: [<tf.Variable 'toy_model/dense/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[1.9967932]], dtype=float32)>, 
#<tf.Variable 'toy_model/dense/bias:0' shape=(1,) dtype=float32, numpy=array([-0.9994885], dtype=float32)>, 
#<tf.Variable 'another_bias:0' shape=() dtype=float32, 초기화 numpy=0.0>]

Initial loss: 9.065
Trainable variables: [<tf.Variable 'toy_model/dense/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[1.9967932]], dtype=float32)>, <tf.Variable 'toy_model/dense/bias:0' shape=(1,) dtype=float32, numpy=array([-0.9994885], dtype=float32)>, <tf.Variable 'another_bias:0' shape=() dtype=float32, numpy=0.0>]


We are only optimizing the additional bias. The weight and bias of toy_model_1 does not change.

In [23]:
# Training loop. anothger bias만 학습
for i in range(300):
    with tf.GradientTape() as tape:
        loss_value = loss(model2, toy_inputs, toy_outputs_2)
    grads = tape.gradient(loss_value, [model2.b]) # gradient w.r.t. `model2.b`, not `model2.trainable_variables`
    optimizer.apply_gradients(zip(grads, [model2.b]), # optimize only `model2.b`
                              global_step=tf.train.get_or_create_global_step())
    if i % 20 == 0:
        print("Loss at step {:03d}: {:.3f}".format(i, loss(model2, toy_inputs, toy_outputs_2)))

print("Final loss: {:.3f}".format(loss(model2, toy_inputs, toy_outputs_2)))
print("Trainable variables:", model2.trainable_variables)

# <tf.Variable 'another_bias:0' shape=() dtype=float32, numpy=2.9930015> +3으로 추가하여서 학습됨

Loss at step 000: 8.709
Loss at step 020: 3.917
Loss at step 040: 1.782
Loss at step 060: 0.830
Loss at step 080: 0.406
Loss at step 100: 0.217
Loss at step 120: 0.133
Loss at step 140: 0.095
Loss at step 160: 0.078
Loss at step 180: 0.071
Loss at step 200: 0.068
Loss at step 220: 0.066
Loss at step 240: 0.065
Loss at step 260: 0.065
Loss at step 280: 0.065
Final loss: 0.065
Trainable variables: [<tf.Variable 'toy_model/dense/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[1.9967932]], dtype=float32)>, <tf.Variable 'toy_model/dense/bias:0' shape=(1,) dtype=float32, numpy=array([-0.9994885], dtype=float32)>, <tf.Variable 'another_bias:0' shape=() dtype=float32, numpy=2.9930015>]


## Wrap-up

So far, we have learned how we can use two types of high-level APIs: Keras and eager execution. For more information about Keras and Eager execution, you can visit https://www.tensorflow.org/guide/keras and https://www.tensorflow.org/guide/eager, respectively (and many other blog posts as well!).